In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
import gc

# Creación features y split de train/validation

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Organización de Datos/TP2/dataset_v2/train.parq', columns=['attack_time', 'watcher_country', 'attacker_country', 'attacker_as_name', 'watcher_as_name', 'attack_type',  'watcher_uuid_enum', 'attacker_ip_enum', 'label' ])

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['watcher_uuid_enum'] = df.watcher_uuid_enum.astype('category')

In [ ]:
df.rename(columns={'attacker_as_name':'attacker', 'watcher_as_name':'watcher', 'watcher_uuid_enum':'watcher_id'}, inplace=True)

In [ ]:
def obtener_rangos_horarios(hora):
    if 0 <= hora < 4:
        return '0-4'
    if 4 <= hora < 8:
        return '4-8'
    if 8 <= hora < 12:
        return '8-12'
    if 12 <= hora < 16:
        return '12-16'
    if 16 <= hora < 20:
        return '16-20'
    else:
        return '20-24'

In [ ]:
df['hora'] = df['attack_time'].dt.hour
df['hour_range'] = df['hora'].map(lambda x: obtener_rangos_horarios(x))
df.drop(['hora'], axis=1,  inplace=True)
df.drop(['attack_time'], axis=1, inplace=True)

In [ ]:
df['attack_protocol'] = df['attack_type'].map(lambda x: x.split(':')[0]).astype('category')
df['attack_type'] = df['attack_type'].map(lambda x: x.split(':')[1]).astype('category')

In [ ]:
most_frequent_attacker_country_attacker = df.groupby('attacker')['attacker_country'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NA))
most_frequent_attacker_country_watcher = df.groupby('watcher')['attacker_country'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NA))
most_frequent_attacker_country_watcherid = df.groupby('watcher_id')['attacker_country'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NA))

In [ ]:
df['attacker_country'].fillna(most_frequent_attacker_country_attacker, inplace=True)
df['attacker_country'].fillna(most_frequent_attacker_country_watcher, inplace=True)
df['attacker_country'].fillna(most_frequent_attacker_country_watcherid, inplace=True)

In [ ]:
del most_frequent_attacker_country_attacker
del most_frequent_attacker_country_watcher
del most_frequent_attacker_country_watcherid

In [ ]:
most_frequent_watcher_country_watcher = df.groupby('watcher')['watcher_country'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NA))
most_frequent_watcher_country_watcherid = df.groupby('watcher_id')['watcher_country'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NA))
most_frequent_watcher_country_attacker = df.groupby('attacker')['watcher_country'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NA))
most_frequent_watcher_country_attackercountry = df.groupby('attacker_country')['watcher_country'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NA))

In [ ]:
df['watcher_country'].fillna(most_frequent_watcher_country_watcher, inplace=True)
df['watcher_country'].fillna(most_frequent_watcher_country_watcherid, inplace=True)
df['watcher_country'].fillna(most_frequent_watcher_country_attacker, inplace=True)
df['watcher_country'].fillna(most_frequent_watcher_country_attackercountry, inplace=True)

In [ ]:
del most_frequent_watcher_country_watcher
del most_frequent_watcher_country_watcherid
del most_frequent_watcher_country_attacker
del most_frequent_watcher_country_attackercountry

In [ ]:
df.drop(['watcher'], axis=1, inplace=True)

In [ ]:
most_frequent_attacker_attackercountry = df.groupby('attacker_country')['attacker'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NA))
df['attacker'].fillna(most_frequent_attacker_attackercountry, inplace=True)

In [ ]:
del most_frequent_attacker_attackercountry

In [ ]:
df['attack_countries'] = df['attacker_country'].astype(str) + '-' + df['watcher_country'].astype(str)
df['attack_countries'] = df['attack_countries'].astype('category')

df['country_hour_range'] = df['hour_range'] + ' - ' + df['watcher_country'].astype(str)
df['country_hour_range'] = df['country_hour_range'].astype('category')

In [ ]:
df.drop(['watcher_country'], axis=1, inplace=True)
df.drop(['hour_range'], axis=1, inplace=True)

In [ ]:
attacker_ips = df['attacker_ip_enum'].unique()

In [ ]:
ips_train, ips_validation = train_test_split(attacker_ips, test_size=0.1, random_state=10)

In [ ]:
train = df[df['attacker_ip_enum'].isin(ips_train)]
validation = df[df['attacker_ip_enum'].isin(ips_validation)]

In [ ]:
train = train.drop(['attacker_ip_enum'], axis=1)
validation = validation.drop(['attacker_ip_enum'], axis=1)

In [ ]:
train.drop_duplicates(inplace=True) #se generan repetidas al dividir time en dia/rango_horario que para entrenar no usaré

In [ ]:
validation = validation.reset_index(drop=True)

In [ ]:
train = train.reset_index(drop=True)

In [ ]:
X_train = train.drop(['label'], axis=1)
y_train = train[['label']]

X_validation = validation.drop(['label'], axis=1)
y_validation = validation[['label']]

In [ ]:
del df
del train
del validation
del attacker_ips
del ips_train
del ips_validation

In [ ]:
gc.collect()

0

# LightGBM

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
clf = lgb.LGBMClassifier()

In [ ]:
params = {
    'n_estimators': [16,24,36,48],
    'num_leaves': [6,8,16,24],
    'max_depth': [4,6,8],
    'random_state' : [10]
    }

In [ ]:
rs = RandomizedSearchCV(clf, param_distributions=params, n_iter=20, random_state=10, scoring='f1')

In [ ]:
rs.fit(X_train, np.ravel(y_train))

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 285171, number of negative: 11218303
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.311152 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 27603
[LightGBM] [Info] Number of data points in the train set: 11503474, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.024790 -> initscore=-3.672213
[LightGBM] [Info] Start training from score -3.672213
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a l

RandomizedSearchCV(estimator=LGBMClassifier(), n_iter=20,
                   param_distributions={'max_depth': [4, 6, 8],
                                        'n_estimators': [16, 24, 36, 48],
                                        'num_leaves': [6, 8, 16, 24],
                                        'random_state': [10]},
                   random_state=10, scoring='f1')

In [ ]:
rs.best_params_

{'random_state': 10, 'num_leaves': 16, 'n_estimators': 24, 'max_depth': 8}

In [ ]:
model = rs.best_estimator_

In [ ]:
model.fit(X_train, np.ravel(y_train))

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 356464, number of negative: 14022879
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.784211 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 27141
[LightGBM] [Info] Number of data points in the train set: 14379343, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.024790 -> initscore=-3.672212
[LightGBM] [Info] Start training from score -3.672212


LGBMClassifier(max_depth=8, n_estimators=24, num_leaves=16, random_state=10)

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
f1_score(y_train, y_pred)

0.7013197262837236

In [ ]:
y_pred = model.predict(X_validation)

In [ ]:
f1_score(y_validation, y_pred)

0.6270386162703862

In [ ]:
X_train.to_pickle("X_train.pkl")
X_validation.to_pickle("X_validation.pkl")
y_train.to_pickle("y_train.pkl")
y_validation.to_pickle("y_validation.pkl")

#unpickled_df = pd.read_pickle("aa.pkl")

In [ ]:
del  X_train
del  X_validation
del  y_train
del  y_validation

# Test

In [ ]:
test = pd.read_parquet('/content/drive/MyDrive/Organización de Datos/TP2/dataset_v2/test.parq', columns=['attack_time', 'watcher_country', 'attacker_country', 'attacker_as_name', 'attack_type',  'watcher_uuid_enum','attacker_ip_enum'])

In [ ]:
test = test.drop_duplicates(subset='attacker_ip_enum').reset_index(drop=True)

In [ ]:
ips_test = test[['attacker_ip_enum']]
test = test.drop(['attacker_ip_enum'], axis=1)

In [ ]:
test['watcher_country'] = test.watcher_country.cat.add_categories('NN').fillna('NN')
test['attacker_as_name'] = test.attacker_as_name.cat.add_categories('NN').fillna('NN')
test['attacker_country'] = test.attacker_country.cat.add_categories('NN').fillna('NN')

test['watcher_uuid_enum'] = test.watcher_uuid_enum.astype('category')

test.rename(columns={'attacker_as_name':'attacker', 'watcher_uuid_enum':'watcher_id'}, inplace=True)

test['hora'] = test['attack_time'].dt.hour
test['hour_range'] = test['hora'].map(lambda x: obtener_rangos_horarios(x))
test.drop(['hora'], axis=1,  inplace=True)
test.drop(['attack_time'], axis=1, inplace=True)

test['attack_protocol'] = test['attack_type'].map(lambda x: x.split(':')[0]).astype('category')
test['attack_type'] = test['attack_type'].map(lambda x: x.split(':')[1]).astype('category')

test['attack_countries'] = test['attacker_country'].astype(str) + '-' + test['watcher_country'].astype(str)
test['attack_countries'] = test['attack_countries'].astype('category')

test['country_hour_range'] = test['hour_range'] + ' - ' + test['watcher_country'].astype(str)
test['country_hour_range'] = test['country_hour_range'].astype('category')

test.drop(['watcher_country'], axis=1, inplace=True)
test.drop(['hour_range'], axis=1, inplace=True)

In [ ]:
y_pred = model.predict(test)

In [ ]:
pred_df = pd.concat([ips_test, pd.DataFrame(y_pred, columns = ['label'])], axis=1)

In [ ]:
pred_df.to_csv('/content/drive/MyDrive/Organización de Datos/TP2/predLGBM.csv', index=False)

In [ ]:
pred_df

,attacker_ip_enum,label
0,7696,0
1,7543,0
2,7280,0
3,1538,0
4,285,0
...,...,...
49415,196304,0
49416,199918,0
49417,192446,0
49418,192056,0


0.5 en Kaggle